In [6]:
import os

In [7]:
%pwd


'c:\\Users\\sheet\\Downloads\\OneDrive\\Desktop\\ML_Project\\Chest-Cancer-Classification-ML-Flow-DVC\\research'

In [8]:
os.chdir("../")


In [9]:
%pwd


'c:\\Users\\sheet\\Downloads\\OneDrive\\Desktop\\ML_Project\\Chest-Cancer-Classification-ML-Flow-DVC'

In [10]:
pip install dagshub

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sheetalgat-hubGit/Chest-Cancer-Classification-ML-Flow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sheetalgat-hubGit"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ba0acb0afe77309e57b59eb42402a42d625db7a3"

In [22]:
import mlflow
print(mlflow.get_tracking_uri())


https://dagshub.com/sheetalgat-hubGit/Chest-Cancer-Classification-ML-Flow-DVC.mlflow


In [23]:
import tensorflow as tf


In [24]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [26]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [27]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/sheetalgat-hubGit/Chest-Cancer-Classification-ML-Flow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [28]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [30]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-26 12:21:43,575: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-26 12:21:43,598: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-26 12:21:43,607: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 25s 4s/step - loss: 0.8725 - accuracy: 0.6176
[2026-01-26 12:22:09,466: INFO: common: json file saved at: scores.json]


2026/01/26 12:22:11 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-26 12:22:13,060: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\sheet\AppData\Local\Temp\tmphsimpfc3\model\data\model\assets
[2026-01-26 12:22:14,045: INFO: builder_impl: Assets written to: C:\Users\sheet\AppData\Local\Temp\tmphsimpfc3\model\data\model\assets]


2026/01/26 12:23:07 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'backports-tarfile'}
Successfully registered model 'VGG16Model'.
2026/01/26 12:35:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
